# ETL - Stations - Precipitation

* Load Raw Data File (`.csv`)
* Create `xarray.Dataset`

In [4]:
import autoroot
from dotenv import load_dotenv
load_dotenv()

from tqdm.auto import tqdm
import xarray as xr
import numpy as np
from st_evt import validate_longitude, validate_latitude

import pint_xarray
from loguru import logger
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Paths

In [27]:
import os
from pathlib import Path

logger.info("Initializaing paths...")

raw_data_dir = Path(os.getenv("RAW_DATA_SAVEDIR"))
clean_data_dir = Path(os.getenv("CLEAN_DATA_SAVEDIR"))

2024-12-04 12:58:43.417 | INFO     | __main__:<module>:4 - Initializaing paths...


In [31]:
logger.info("Opening dataset...")

logger.debug(f"Raw Data: {raw_data_dir}")
ds_ = xr.open_dataset(raw_data_dir.joinpath("pr.nc"))

assert ds_.pr.shape[:2] == (2_407, 22_645)
logger.debug(f"Shape: {ds_.pr.shape}")

2024-12-04 12:59:51.305 | INFO     | __main__:<module>:1 - Opening dataset...
2024-12-04 12:59:51.306 | DEBUG    | __main__:<module>:3 - Raw Data: /home/juanjohn/pool_data/dynev4eo/data/raw
2024-12-04 12:59:51.313 | DEBUG    | __main__:<module>:7 - Shape: (2407, 22645)


## Temperature

### Load Dataframes

#### Coordinates

In [12]:
logger.info("Loading stations...")

# Load Station Coordinates
df_coords = pd.read_csv(raw_data_dir.joinpath("ubicacion_estaciones_spain.csv"), delimiter=";", index_col=0, decimal=",")

logger.info("Checking Coords shapes...")
assert df_coords.shape == (5238, 5)
logger.debug(f"Shape: {df_coords.shape}")
logger.info("Checking Column names...")
columns = ["name", "alt", 'lon', "lat", "prov"]
assert set(columns).issubset(df_coords.columns)

2024-12-04 12:57:05.419 | INFO     | __main__:<module>:1 - Loading stations...
2024-12-04 12:57:05.450 | INFO     | __main__:<module>:6 - Checking Coords shapes...
2024-12-04 12:57:05.451 | DEBUG    | __main__:<module>:8 - Shape: (5238, 5)
2024-12-04 12:57:05.451 | INFO     | __main__:<module>:9 - Checking Column names...


#### Precipitation

In [13]:
logger.info("Loading precipitation...")
# Load Max Temperature Values
df_precip = pd.read_csv(raw_data_dir.joinpath("pr.csv"), index_col=0)

2024-12-04 12:57:11.071 | INFO     | __main__:<module>:1 - Loading precipitation...


In [14]:
logger.info("Checking Coords shapes...")
assert df_precip.shape == (22_645, 2_448)

2024-12-04 12:57:14.015 | INFO     | __main__:<module>:1 - Checking Coords shapes...


### Create XArray Dataset

In [15]:
logger.info("Creating empty dataset...")
coordinates = dict(
    station_id=list(),
    station_name=list(),
    lat=list(),
    lon=list(),
    alt=list(),
    values=list()
)

2024-12-04 12:57:46.653 | INFO     | __main__:<module>:1 - Creating empty dataset...


In [16]:
xr_datasets = xr.Dataset()
pbar = tqdm(df_precip.columns, leave=True)
for iname in pbar:

    try:
        ids = df_precip[str(iname)]
        icoords = df_coords.loc[str(iname)]
        # extract coordinates
        coordinates["station_id"].append(icoords.name)
        coordinates["station_name"].append(icoords["name"].lower())
        coordinates["lat"].append(np.float32(icoords["lat"]))
        coordinates["lon"].append(np.float32(icoords["lon"]))
        coordinates["alt"].append(np.float32(icoords["alt"]))
        coordinates["values"].append(np.float32(ids.values))
    except KeyError:
        pass

ds_precip = xr.Dataset(
    {
        "pr": (("station_id", "time"), coordinates['values']),
        "lon": (("station_id"), coordinates['lon']),
        "lat": (("station_id"), coordinates['lat']),
        "alt": (("station_id"), coordinates['alt']),
        "station_name": (("station_id"), coordinates['station_name']),
    },
    coords={
        "station_id": coordinates["station_id"],
        "time": ds_.time
    }
)

logger.info(f"Cleaning metadata and coordinates...")

# assign coordinates
ds_precip = ds_precip.set_coords(["lon", "lat", "alt", "station_name"])
ds_precip


  0%|          | 0/2448 [00:00<?, ?it/s]

2024-12-04 12:57:50.546 | INFO     | __main__:<module>:32 - Cleaning metadata and coordinates...


<xarray.Dataset> Size: 109MB
Dimensions:       (station_id: 1200, time: 22645)
Coordinates:
    lon           (station_id) float32 5kB 0.87 1.049 1.162 ... -17.89 -18.1
    lat           (station_id) float32 5kB 40.96 41.14 41.14 ... 27.82 27.74
    alt           (station_id) float32 5kB 32.0 124.0 71.0 ... 620.0 32.0 299.0
    station_name  (station_id) <U50 240kB 'vandellòs' ... 'sabinosa'
  * time          (time) datetime64[ns] 181kB 1961-01-01 ... 2022-12-31
  * station_id    (station_id) <U5 24kB '0002I' '0015I' ... 'C929I' 'C939U'
Data variables:
    pr            (station_id, time) float32 109MB 0.0 0.0 2.5 ... 0.0 0.0 0.0

In [17]:
logger.info("Checking xarray.dataset shape...")
assert ds_precip.pr.shape == (1_200, 22_645)

2024-12-04 12:57:52.753 | INFO     | __main__:<module>:1 - Checking xarray.dataset shape...


In [18]:
ds_precip = ds_precip.drop_duplicates(dim="station_id")

logger.info("Checking xarray.dataset shape...")
assert ds_precip.pr.shape == (1_200, 22_645)

2024-12-04 12:57:53.215 | INFO     | __main__:<module>:3 - Checking xarray.dataset shape...


### Correct Coordinates and Units

In [20]:
logger.info("Validating Coordinates...")

# valudate coordinates
ds_precip = validate_longitude(ds_precip)
ds_precip = validate_latitude(ds_precip)

# sort by time
ds_precip = ds_precip.sortby("time")

# clean up attributes names
ds_precip["pr"].attrs["standard_name"] = "daily_cumulative_precipitation"
ds_precip["pr"].attrs["long_name"] = "Daily Cumulative Precipitation"


ds_precip["alt"].attrs["standard_name"] = "altitude"
ds_precip["alt"].attrs["long_name"] = "Altitude"

# validate units
ds_precip = ds_precip.pint.quantify(
    {"pr": "mm / day", 
    "lon": "degree", 
    "lat": "degree",
    "alt": "meters"
    }
)
ds_precip = ds_precip.pint.dequantify()

ds_precip 

2024-12-04 12:58:05.278 | INFO     | __main__:<module>:1 - Validating Coordinates...


<xarray.Dataset> Size: 109MB
Dimensions:       (station_id: 1200, time: 22645)
Coordinates:
    lon           (station_id) float32 5kB 0.87 1.049 1.162 ... -17.89 -18.1
    lat           (station_id) float32 5kB 40.96 41.14 41.14 ... 27.82 27.74
    alt           (station_id) float32 5kB 32.0 124.0 71.0 ... 620.0 32.0 299.0
    station_name  (station_id) <U50 240kB 'vandellòs' ... 'sabinosa'
  * time          (time) datetime64[ns] 181kB 1961-01-01 ... 2022-12-31
  * station_id    (station_id) <U5 24kB '0002I' '0015I' ... 'C929I' 'C939U'
Data variables:
    pr            (station_id, time) float32 109MB 0.0 0.0 2.5 ... 0.0 0.0 0.0

### Good Stations

In [21]:
# Load the GOOD Stations
red_feten_stations = pd.read_csv(raw_data_dir.joinpath("red_feten.csv"))
red_feten_stations.shape

(1178, 1)

In [22]:
red_feten_stations = np.intersect1d(red_feten_stations.id, ds_precip.station_id)

logger.info(f"# Red Feten Stations: {len(red_feten_stations)}...")

2024-12-04 12:58:17.941 | INFO     | __main__:<module>:3 - # Red Feten Stations: 146...


In [23]:
# create mask
red_feten_mask = ds_precip.station_id.isin(red_feten_stations).rename("red_feten").astype(np.int8)

# assign as coordinates
ds_precip = ds_precip.assign_coords({"red_feten_mask": red_feten_mask})

ds_precip

<xarray.Dataset> Size: 109MB
Dimensions:         (station_id: 1200, time: 22645)
Coordinates:
    lon             (station_id) float32 5kB 0.87 1.049 1.162 ... -17.89 -18.1
    lat             (station_id) float32 5kB 40.96 41.14 41.14 ... 27.82 27.74
    alt             (station_id) float32 5kB 32.0 124.0 71.0 ... 32.0 299.0
    station_name    (station_id) <U50 240kB 'vandellòs' ... 'sabinosa'
  * time            (time) datetime64[ns] 181kB 1961-01-01 ... 2022-12-31
  * station_id      (station_id) <U5 24kB '0002I' '0015I' ... 'C929I' 'C939U'
    red_feten_mask  (station_id) int8 1kB 1 0 1 1 0 0 1 0 0 ... 0 0 1 0 0 0 1 0
Data variables:
    pr              (station_id, time) float32 109MB 0.0 0.0 2.5 ... 0.0 0.0 0.0

In [24]:
ds_precip = ds_precip.drop_duplicates(dim="station_id")
ds_precip

<xarray.Dataset> Size: 109MB
Dimensions:         (station_id: 1200, time: 22645)
Coordinates:
    lon             (station_id) float32 5kB 0.87 1.049 1.162 ... -17.89 -18.1
    lat             (station_id) float32 5kB 40.96 41.14 41.14 ... 27.82 27.74
    alt             (station_id) float32 5kB 32.0 124.0 71.0 ... 32.0 299.0
    station_name    (station_id) <U50 240kB 'vandellòs' ... 'sabinosa'
  * time            (time) datetime64[ns] 181kB 1961-01-01 ... 2022-12-31
  * station_id      (station_id) <U5 24kB '0002I' '0015I' ... 'C929I' 'C939U'
    red_feten_mask  (station_id) int8 1kB 1 0 1 1 0 0 1 0 0 ... 0 0 1 0 0 0 1 0
Data variables:
    pr              (station_id, time) float32 109MB 0.0 0.0 2.5 ... 0.0 0.0 0.0

## Save

In [30]:
logger.info(f"Saving data to disk...")

save_name = "pr_stations_spain.zarr"
full_save_path = clean_data_dir.joinpath(save_name)

logger.debug(f"Save Path: {full_save_path}")
ds_precip.to_zarr(full_save_path, mode="w")

2024-12-04 12:59:32.874 | INFO     | __main__:<module>:1 - Saving data to disk...
2024-12-04 12:59:32.875 | DEBUG    | __main__:<module>:6 - Save Path: /home/juanjohn/pool_data/dynev4eo/data/clean/pr_stations_spain.zarr
